In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glioblastoma"
cohort = "GSE148949"

# Input paths
in_trait_dir = "../../input/GEO/Glioblastoma"
in_cohort_dir = "../../input/GEO/Glioblastoma/GSE148949"

# Output paths
out_data_file = "../../output/preprocess/Glioblastoma/GSE148949.csv"
out_gene_data_file = "../../output/preprocess/Glioblastoma/gene_data/GSE148949.csv"
out_clinical_data_file = "../../output/preprocess/Glioblastoma/clinical_data/GSE148949.csv"
json_path = "../../output/preprocess/Glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"BKM120 Treated WHIMs_17 Model Cohort"
!Series_summary	"Aberrant activation of PI3K pathway is frequently observed in triple negative breast cancer (TNBC).  However single agent PI3K inhibitors have shown modest anti-tumor activity.  To investigate biomarkers of response, we tested 17 TNBC PDX models with diverse genetic and proteomic background, with varying PI3K pathway signaling activities for their tumor growth response to the pan-PI3K inhibitor BKM120 as well as baseline and treatment induced proteomic changes as assessed by reverse phase protein array (RPPA).  We demonstrated that PI3K inhibition induces varying degrees of tumor growth inhibition (TGI), with 5 models demonstrating over 80% TGI. BKM120 consistently reduced PI3K pathway activity as demonstrated by reduced pAKT following therapy.  Several biomarkers showed significant association with resistance, including baseline levels of growth factor receptors (EGFR, pHER3 Y1197), PI3Kp85 r

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset and extract clinical features

# 1. Determine Gene Expression Data Availability
# Based on the background info, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary
# 2.1 Data Availability

# For trait (Glioblastoma): 
# The data mentions "5_Glioblastoma, brain" in the tissue list
trait_row = 0  # The information is in key 0

# For age: No age information available
age_row = None

# For gender: No gender information available
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait information to binary"""
    if pd.isna(value):
        return None
    
    # Extract text after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if Glioblastoma is mentioned in the cell
    if "Glioblastoma" in value or "glioblastoma" in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age information (not available in this dataset)"""
    return None

def convert_gender(value):
    """Convert gender information (not available in this dataset)"""
    return None

# 3. Save Metadata - Initial filtering
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Prepare the clinical data DataFrame
    # Since we only have a dictionary with one key, we'll convert it to a DataFrame
    clinical_data = pd.DataFrame(
        {0: ['tissue: Total RNA from 10 human cell lines: 1_Adenocarcinoma, mammary gland 2_Hepatoblastoma, liver 3_Adenocarcinoma, cervix 4_Embryonal carcinoma, testis 5_Glioblastoma, brain 6_Melanoma 7_Liposarcoma 8_Histiocytic Lymphoma; macrophage; histocyte 9_ Lymphoblastic leukemia, T lymphoblast 10_Plasmacytoma; myeloma; B lymphocyte.  Also, mRNA spiked in from MCF7 and ME16C.']},
        index=['sample_characteristics']
    ).T
    
    # Extract clinical features using geo_select_clinical_features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview_result = preview_df(selected_clinical_df)
    print("Preview of processed clinical data:")
    print(preview_result)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of processed clinical data:
{'sample_characteristics': [1.0]}
Clinical data saved to: ../../output/preprocess/Glioblastoma/clinical_data/GSE148949.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 76
Header line: "ID_REF"	"GSM4486560"	"GSM4486561"	"GSM4486562"	"GSM4486563"	"GSM4486564"	"GSM4486565"	"GSM4486566"	"GSM4486567"	"GSM4486568"	"GSM4486569"	"GSM4486570"	"GSM4486571"	"GSM4486572"	"GSM4486573"	"GSM4486574"	"GSM4486575"	"GSM4486576"	"GSM4486577"	"GSM4486578"	"GSM4486579"	"GSM4486580"	"GSM4486581"	"GSM4486582"	"GSM4486583"	"GSM4486584"	"GSM4486585"	"GSM4486586"	"GSM4486587"	"GSM4486588"	"GSM4486589"	"GSM4486590"	"GSM4486591"	"GSM4486592"	"GSM4486593"	"GSM4486594"	"GSM4486595"	"GSM4486596"	"GSM4486597"	"GSM4486598"	"GSM4486599"	"GSM4486600"	"GSM4486601"	"GSM4486602"	"GSM4486603"	"GSM4486604"	"GSM4486605"	"GSM4486606"	"GSM4486607"	"GSM4486608"	"GSM4486609"	"GSM4486610"	"GSM4486611"	"GSM4486612"	"GSM4486613"
First data line: "1/2-SBSRNA4"	0.33017439	0.201567061	0.333105343	0.328331502	0.399451105	0.091421952	0.739362035	1.227083303	0.55493156	0.670595195	0.652113571	0.661813238	0.073208376	0.156654784	0.460903565	0.696163629	0.309558797	0.691537309	0.

Index(['1/2-SBSRNA4', '41334', '41335', '41336', '41337', '41338', '41339',
       '41340', '41341', '41342', '41343', '41344', '41518', '41519', '41520',
       '41521', '41522', '41523', '41524', '41525'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers provided in the data
# The first few identifiers are:
# '1/2-SBSRNA4', '41334', '41335', '41336', '41337', etc.

# '1/2-SBSRNA4' appears to be a small non-coding RNA identifier
# The numerical identifiers (41334, 41335, etc.) are likely probe IDs from a microarray platform
# These are not standard human gene symbols (like BRCA1, TP53, etc.)

# Therefore, these identifiers require mapping to standard gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE148949
Line 6: !Series_title = BKM120 Treated WHIMs_17 Model Cohort
Line 7: !Series_geo_accession = GSE148949
Line 8: !Series_status = Public on Dec 31 2020
Line 9: !Series_submission_date = Apr 20 2020
Line 10: !Series_last_update_date = Dec 26 2023
Line 11: !Series_pubmed_id = 33371187
Line 12: !Series_summary = Aberrant activation of PI3K pathway is frequently observed in triple negative breast cancer (TNBC).  However single agent PI3K inhibitors have shown modest anti-tumor activity.  To investigate biomarkers of response, we tested 17 TNBC PDX models with diverse genetic and proteomic background, with varying PI3K pathway signaling activities for their tumor growth response to the pan-PI3K inhibitor

### Step 6: Gene Identifier Mapping

In [7]:
# Let's first properly load both the gene expression data and gene annotation data
# 1. Load gene expression data from the matrix file
gene_expr_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_expr_data.shape}")

# 2. Load gene annotation data from the SOFT file
# Based on the preview, it looks like the SOFT file already contains proper gene symbols in both ID and ORF columns
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# 3. Check which columns in gene_annotation contain the identifiers and gene symbols
# From the preview, 'ID' appears to contain gene symbols directly
print("\nGene annotation column names:")
print(gene_annotation.columns.tolist())
print("\nFirst few rows of gene annotation:")
print(gene_annotation.head())

# 4. Create the mapping dataframe
# Since both ID and ORF appear to contain gene symbols, and the gene expression data has the same type of ID,
# we can directly map them
mapping_df = gene_annotation[['ID', 'ORF']].copy()
mapping_df.rename(columns={'ORF': 'Gene'}, inplace=True)
print("\nGene mapping dataframe preview:")
print(mapping_df.head())

# 5. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_expr_data, mapping_df)
print(f"\nGene expression data after mapping: {gene_data.shape}")
print("\nFirst few genes after mapping:")
print(gene_data.index[:10])

# 6. Save the gene expression data for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene expression data shape: (29657, 54)


Gene annotation data shape: (1631189, 2)

Gene annotation column names:
['ID', 'ORF']

First few rows of gene annotation:
         ID       ORF
0      A1BG      A1BG
1  A1BG-AS1  A1BG-AS1
2      A1CF      A1CF
3     A2LD1     A2LD1
4       A2M       A2M

Gene mapping dataframe preview:
         ID      Gene
0      A1BG      A1BG
1  A1BG-AS1  A1BG-AS1
2      A1CF      A1CF
3     A2LD1     A2LD1
4       A2M       A2M



Gene expression data after mapping: (18460, 54)

First few genes after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Glioblastoma/gene_data/GSE148949.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using data from previous step
# We already have gene_data from the previous step
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Generate clinical data
# Based on previous output, we know we only have the trait information
# All samples are glioblastoma as indicated by the tissue info in sample characteristics

# First, let's create a proper clinical dataframe based on what we know
# We need a dataframe with sample IDs as columns and trait as the only row
clinical_df = pd.DataFrame(index=[trait], 
                          data={col: 1 for col in normalized_gene_data.columns})
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")

cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
is_trait_biased = False
if len(cleaned_data) > 0:
    trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    is_trait_biased = trait_biased
else:
    print("No data remains after handling missing values.")
    is_trait_biased = True

# 6. Final validation and save
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=len(normalized_gene_data) > 0, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=cleaned_data,
    note=f"Dataset contains gene expression data for {trait} analysis. No demographic features available."
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (18221, 54)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC']


Normalized gene data saved to ../../output/preprocess/Glioblastoma/gene_data/GSE148949.csv
Clinical data shape: (1, 54)
Clinical data preview:
              GSM4486560  GSM4486561  GSM4486562  GSM4486563  GSM4486564  \
Glioblastoma           1           1           1           1           1   

              GSM4486565  GSM4486566  GSM4486567  GSM4486568  GSM4486569  ...  \
Glioblastoma           1           1           1           1           1  ...   

              GSM4486604  GSM4486605  GSM4486606  GSM4486607  GSM4486608  \
Glioblastoma           1           1           1           1           1   

              GSM4486609  GSM4486610  GSM4486611  GSM4486612  GSM4486613  
Glioblastoma           1           1           1           1           1  

[1 rows x 54 columns]
Clinical data saved to ../../output/preprocess/Glioblastoma/clinical_data/GSE148949.csv
Linked data shape: (54, 18222)
Linked data preview (first 5 rows, first 5 columns):
            Glioblastoma      A1BG  A1BG-AS

Data shape after handling missing values: (54, 18222)
Quartiles for 'Glioblastoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Glioblastoma' in this dataset is severely biased.

Data was determined to be unusable or empty and was not saved
